In [ ]:
# standard loads
import numpy as np #type:ignore
import matplotlib.pyplot as plt #type:ignore
from matplotlib.colors import ListedColormap #type:ignore
import pandas as pd #type:ignore
import os
import json
import h5py #type:ignore
from skimage import morphology #type:ignore
from skimage.morphology import skeletonize #type:ignore
from skimage.morphology import dilation, disk # type: ignore
from scipy import ndimage # type: ignore
import time
import warnings
warnings.filterwarnings("ignore") # ignore warnings


# import specific algorithm functions
from input_formatting import input_formatting 
from readvardef import readvardef
from Loadlidarmaps import Loadlidarmaps
from RemoveaboveHAT import RemoveaboveHAT
from Creek_Repair import process_creek_mask, save_creek_mask_h5, load_creek_mask_h5, process_creek_mask_diagnostic
from Creek_Ordering import figure_creek_skeleton, process_creek_ordering, process_creek_ordering_diagnostic
from check_plots_step5 import plot_creek_orders, plot_creek_orders_big
from Strahler_Correction_class import CreekNetworkAnalyzer
from Display_Orders import process_creek_orders
from Process_CrossSections import process_xsects, process_xsects_diagnostic
from Process_JunctionAngles import process_junction_angles, process_junction_angles_diagnostic
from Save_Results import process_creek_morphometry, process_creek_morphometry_diagnostic

%config InlineBackend.figure_format='retina' # hig-res plots for a Retina display, uncomment while working

In [ ]:
# User Input filenames
# manual input, as opposed to MATLAB script's GUI selection

basename_elev = 'otc_DEM_converted_m.asc' 
basename_slope = 'otc_slope_degrees.asc'
basename_UserVar = 'UserVar_otc.txt' 

filepath_input_folder = '/Users/sam/RDE Dropbox/Projects/22386.Ashleyville Marsh/WORK/CODE/Python/CHIROL_CREEK_EXTRACTION_ALGORITHM/INPUTS/'

filename_elev = filepath_input_folder+basename_elev
filename_slope = filepath_input_folder+basename_slope
filename_UserVar = filepath_input_folder+basename_UserVar

## input_formatting

In [ ]:
# input_formatting
# formats elevation and slope raster ascii files in to .h5
# requires user manual input for filename of elevation and slope

from input_formatting import input_formatting

# check if OUTPUTS folder is created - if not, create one
folder_path_check = "OUTPUTS"

# Check if the folder exists
if not os.path.exists(folder_path_check):
    # Create the folder
    os.makedirs(folder_path_check)
    print(f"Folder '{folder_path_check}' created.")
else:
    print(f"Folder '{folder_path_check}' already exists.")

print('Elevation Files:')
# Change file names for specific site - i.e., filename = 'INPUTS/blahblah_elev.txt' or 'INPUTS/blahblah_slope.txt'
filename_out_elev, filename_out_meta_elev = input_formatting(filename_elev)
print('File was saved as: ', filename_out_elev)
print('Metadata file was saved as: ', filename_out_meta_elev)

print('')

print('Slope Files:')
filename_out_slope, filename_out_meta_slope = input_formatting(filename_slope)
print('File was saved as: ', filename_out_slope)
print('Metadata file was saved as: ', filename_out_meta_slope)

## CHIROL_CREEK_ALGORITHM

In [ ]:
# Section 1 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI and ChatGPT, reviewed by Sam Kraus 2024/09/13

# import specific algorithm functions
from readvardef import readvardef

start_time = time.time()

# 1) User to select input variables
figview = 0  # Change to 1 to see additional figures

# Open User Variables instruction file
result = readvardef('INPUTS/'+basename_UserVar)
if result is None:
    print("User aborted or error occurred.")
    exit()

FILENAMES, PROCESSING, TIDE, THRESHOLDS, RECONNECTION = result

# Unpack data from global variables
name = FILENAMES["name"]
shortname = FILENAMES["shortname"]
elev = FILENAMES["elev"]
elevmeta = FILENAMES["elevmeta"]
slope = FILENAMES["slope"]
slopemeta = FILENAMES["slopemeta"]

resamplestep = PROCESSING["resamplestep"]
threshold = PROCESSING["threshold"]
detrendyn = PROCESSING["detrendyn"]
outletdetection = PROCESSING["outletdetection"]

HAT = TIDE["HAT"]
MHWS = TIDE["MHWS"]
MHWN = TIDE["MHWN"]
MLWS = TIDE["MLWS"]
MLWN = TIDE["MLWN"]

Cth = THRESHOLDS["Cth"]
Ctharea = THRESHOLDS["Ctharea"]
LZth = THRESHOLDS["LZth"]
LZtharea = THRESHOLDS["LZtharea"]
HZth = THRESHOLDS["HZth"]
HZtharea = THRESHOLDS["HZtharea"]

nbbreaches = RECONNECTION["nbbreaches"]
noisethreshold = RECONNECTION["noisethreshold"]
reconnect = RECONNECTION["reconnect"]
reconnectiondist = RECONNECTION["reconnectiondist"]
ordermax = RECONNECTION["ordermax"]
filtersmall1 = RECONNECTION["filtersmall1"]
filterlarge1 = RECONNECTION["filterlarge1"]
smoothing = RECONNECTION["smoothing"]
connectivity = RECONNECTION["connectivity"]
holesizeinfill = RECONNECTION["holesizeinfill"]
filtersmall2 = RECONNECTION["filtersmall2"]
filterlarge2 = RECONNECTION["filterlarge2"]

#  Print out the values to verify they were read correctly
# print("Loaded variables:")
# for category in [FILENAMES, PROCESSING, TIDE, THRESHOLDS, RECONNECTION]:
#     for key, value in category.items():
#         print(f"{key}: {value}")

end_time = time.time()
# print(f"Execution time: {end_time - start_time} seconds")

In [ ]:
# Section 2 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI and ChatGPT, reviewed by Sam Kraus 2024/09/15

# import specific algorithm functions
from Loadlidarmaps import Loadlidarmaps
from RemoveaboveHAT import RemoveaboveHAT

# Load the data
Xs, Ys, Zs, X, Y, Z, gs, xc, yc = Loadlidarmaps(elev, elevmeta, slope, slopemeta)

# Remove values above HAT
Z, Zs, meandepth, meanslope = RemoveaboveHAT(Z, Zs, HAT, detrendyn)

# Print some information about the loaded and processed data
print(f"Data loaded and processed:")
print(f"X range: {np.min(X)} to {np.max(X)} meters")
print(f"Y range: {np.min(Y)} to {np.max(Y)} meters")
print(f"Z range: {np.nanmin(Z)} to {np.nanmax(Z)} meters")
print(f"Mean depth: {meandepth} meters")
print(f"Mean slope: {meanslope} degrees")
print(f"Grid resolution: {gs} meters")

# Subsample the dataset to make mapping quicker
# X2 = X[::resamplestep, ::resamplestep]
# Y2 = Y[::resamplestep, ::resamplestep]
X2 = X[::resamplestep] # was 2D in MATLAB, I guess not 2D in Python?
Y2 = Y[::resamplestep] # was 2D in MATLAB, I guess not 2D in Python?
Z2 = Z[::resamplestep, ::resamplestep]

print(f"Subsampled data shape: {Z2.shape}")

In [ ]:
# Section 3 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI and ChatGPT, reviewed by Sam Kraus 2024/09/15 and 2024/10/10

# Your variables
variables_detection = {
    "threshold": 1,
    "Zs": Zs.tolist(),   # Convert arrays to lists to make them JSON serializable
    "Z": Z.tolist(),
    "gs": gs,
    "X": X,
    "Y": Y,
    "X2": X2,
    "Y2": Y2,
    "Z2": Z2,
    "LZtharea": LZtharea,
    "LZth": LZth,
    "HZtharea": HZtharea,
    "HZth": HZth,
    "Ctharea": Ctharea,
    "Cth": Cth,
    "shortname": shortname
}

# Convert all numpy arrays to lists
variables_detection = {key: value.tolist() if isinstance(value, np.ndarray) else value for key, value in variables_detection.items()}

# Save to a JSON file
with open("variables_Creek_Detection.json", "w") as f:
    json.dump(variables_detection, f)

# After running this cell, you need to run in terminal, following these steps: <br>
# 1) If you're on macOS or Linux, make sure that Tkinter (which matplotlib uses for plotting) is properly installed. 
#     On macOS, you can install it via Homebrew using: <br>
#     ```brew install python-tk```
#     NOTE: Homebrew is a package manager for macOS (and Linux)
#       that simplifies the process of installing, updating, and managing software and tools on your system.
# 2) Open terminal (Command Prompt in Windows)
# 3) Change current directory to folder containing creek extraction .py files - the "~" is to reference your user local folder
#     (spaces in filepath need to be preceded by a single backslash \ to not break up continuous filepath): <br>
#      ```cd ~/RDE\ Dropbox/Projects/22386.Ashleyville\ Marsh/WORK/CODE/Python/CHIROL_CREEK_EXTRACTION_ALGORITHM```
# 4) Run .py file:  <br>
#     ```python Creek_Detection.py```
# 5) Plots should appear in python GUI windows - you can adjust plot positioning in this window, and save the figures. 
#       If the threshold value is 1, you will be prompted to click GUI plots of slope and hypsometry.
# 6) As instructed in terminal print statement, press Enter (return) key to close plots.

In [ ]:
# Load the Step 3 processed variables from the JSON file
with open(f"OUTPUTS/{shortname}_processed_variables_Creek_Detection.json", "r") as f:
    variables = json.load(f)

creek = variables["creek"]
CumArea = variables["CumArea"]
binrange = variables["binrange"]
CumAreas = variables["CumAreas"]
binranges = variables["binranges"]

In [ ]:
# Section 4 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI and ChatGPT, reviewed by Sam Kraus 2024/10/14

# import specific algorithm functions
from Creek_Repair import process_creek_mask, save_creek_mask_h5, load_creek_mask_h5, process_creek_mask_diagnostic

# load raw creek mask from Step 3
with h5py.File(f'OUTPUTS/{shortname}_creekmask_raw.h5', 'r') as hf:
    creekmask_raw = hf['creekmask_raw'][:]
    
# execute functions in Creek_Repair.py and process_creek_mask function
creekmask = process_creek_mask(creekmask_raw, reconnect, noisethreshold, reconnectiondist, connectivity,
                                    filtersmall1, filterlarge1, smoothing, filtersmall2, filterlarge2, holesizeinfill)
# ^the process_creek_mask function inputs differ from the .py file due to use of underscores in .py as opposed to no underscores in .ipynb
save_creek_mask_h5(creekmask, 'OUTPUTS/{}_creekmask.h5'.format(shortname))

# # for troubleshooting issues in oversmoothing and filling operations, run diagnostic function with helpful print statements and plots:
# repaired_mask = process_creek_mask_diagnostic(creekmask_raw, reconnect, noisethreshold, reconnectiondist, connectivity,
#                                     filtersmall1, filterlarge1, smoothing, filtersmall2, filterlarge2, holesizeinfill)

In [ ]:
# Section 5 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI, reviewed line-by-line by Sam Kraus 2024/11/25

# import specific algorithm functions
from Creek_Ordering import figure_creek_skeleton, process_creek_ordering, process_creek_ordering_diagnostic

# load raw creek mask from Step 4
creekmask = load_creek_mask_h5(f'OUTPUTS/{shortname}_creekmask.h5')

# prep data for functions
skeleton = morphology.skeletonize(creekmask)

# # execute functions in Creek_Ordering.py
# STRAHLER, STRAIGHTDIST, IDXSEG, IDXBRANCH, idxbreach, xbreach, ybreach, skeleton_breached, creekorder, creekordersing, PTS, ID = process_creek_ordering(ordermax, Z, skeleton, outletdetection, nbbreaches)

# diagnostic version - uncomment and run for debugging:
results_ordering = process_creek_ordering_diagnostic(ordermax, Z, skeleton, outletdetection, nbbreaches)
STRAHLER, STRAIGHTDIST, IDXSEG, IDXBRANCH, idxbreach, xbreach, ybreach, skeleton_breached, creekorder, creekordersing, PTS, ID = results_ordering

In [ ]:
from check_plots_step5 import plot_creek_orders, plot_creek_orders_big

# Plot the sample data for skeleton
fig, ax = plot_creek_orders(skeleton_breached, creekordersing)
plt.show()

# Plot the sample data
fig, ax = plot_creek_orders_big(skeleton_breached, creekorder, ordermax, X=None, Y=None, colors=None, figsize=(10, 8), dpi=100)
plt.show()

In [ ]:
# Section 6 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI, reviewed by Sam Kraus 2024/12/15
from Strahler_Correction_class import CreekNetworkAnalyzer

# Your variables
variables_correction = {
    "skeleton": skeleton_breached,
    "X": X,
    "Y": Y,
    "creek_order": creekorder,
    "creek_order_single": creekordersing,
    "pts": PTS,
    "order_max": ordermax,
    "STRAHLER": STRAHLER,
    "STRAIGHTDIST": STRAIGHTDIST,
    "IDXBRANCH": IDXBRANCH,
    "IDXSEG": IDXSEG,
    "shortname": shortname
}

# Convert all numpy arrays to lists
variables_correction = {key: value.tolist() if isinstance(value, np.ndarray) else value for key, value in variables_correction.items()}

# Save to a JSON file
with open("variables_Strahler_Correction.json", "w") as f:
    json.dump(variables_correction, f)

# After running this cell, you need to run in terminal, following these steps: <br>
# 1) If you're on macOS or Linux, make sure that Tkinter (which matplotlib uses for plotting) is properly installed. 
#     On macOS, you can install it via Homebrew using: <br>
#     ```brew install python-tk```
#     NOTE: Homebrew is a package manager for macOS (and Linux)
#       that simplifies the process of installing, updating, and managing software and tools on your system.
# 2) Open terminal (Command Prompt in Windows)
# 3) Change current directory to folder containing creek correction .py files - the "~" is to reference your user local folder
#     (spaces in filepath need to be preceded by a single backslash \ to not break up continuous filepath): <br>
#      ```cd ~/RDE\ Dropbox/Projects/22386.Ashleyville\ Marsh/WORK/CODE/Python/CHIROL_CREEK_EXTRACTION_ALGORITHM```
# 4) Run .py file:  <br>
#     ```python Strahler_Correction_execute.py```
# 5) Plots should appear in python GUI windows - you can adjust plot positioning in this window, and save the figures. 
#       Follow the GUI instructions to correct loops.
# 6) Click the "Finish Correction" button after finishing ALL corrections

In [ ]:
# Section 7 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI, reviewed by Sam Kraus 12/16/24

from Display_Orders import process_creek_orders

# Load the variables from the JSON file
with open(f"OUTPUTS/{shortname}_processed_variables_Strahler_Correction.json", "r") as f:
    variables = json.load(f)

skeleton = variables["skeleton"]
X = variables["X"]
Y = variables["Y"]
creek_order = variables["creek_order"]
creek_order_single = variables["creek_order_single"]
creek_order_swapped = variables["creek_order_swapped"]
creek_order_single_swapped = variables["creek_order_single_swapped"]
PTS = variables["PTS"]
order_max = variables["order_max"]
STRAHLER = variables["STRAHLER"]
# print('STRAHLER = ', STRAHLER)
STRAIGHTDIST = variables["STRAIGHTDIST"]
IDXBRANCH = variables["IDXBRANCH"]
IDXSEG = variables["IDXSEG"]

def convert_to_numpy(data, dtype=None):
    """Safely convert input data to numpy array"""
    try:
        if isinstance(data, list):
            # Handle nested lists by converting inner lists first
            if data and isinstance(data[0], list):
                data = [np.array(row) for row in data]
        return np.array(data, dtype=dtype)
    except Exception as e:
        print(f"Error converting data to numpy array: {e}")
        print("Data sample:", data[:2] if isinstance(data, list) else data)
        raise

# Convert to numpy arrays first
skeleton = convert_to_numpy(skeleton, dtype=bool)
X = convert_to_numpy(X, dtype=float)
Y = convert_to_numpy(Y, dtype=float)
X = X - np.nanmin(X)
Y = Y - np.nanmin(Y)
creek_order = convert_to_numpy(creek_order, dtype=float)
creek_order_single = convert_to_numpy(creek_order, dtype=float)
creek_order_swapped = convert_to_numpy(creek_order_swapped, dtype=float)
creek_order_single_swapped = convert_to_numpy(creek_order_single_swapped, dtype=float)
PTS = convert_to_numpy(PTS, dtype=bool)
order_max = int(order_max)
STRAHLER = convert_to_numpy(STRAHLER, dtype=float)
# print('STRAHLER numpy = ', STRAHLER)
STRAIGHTDIST = convert_to_numpy(STRAIGHTDIST, dtype=float)
IDXBRANCH = convert_to_numpy(IDXBRANCH, dtype=float)
IDXSEG = convert_to_numpy(IDXSEG, dtype=float)

# Execute loaded Step 7 function
SINUOSITY, STRAIGHTDIST, STRAHLER, SEGMENTS, SINUOUSLENGTH = process_creek_orders(STRAHLER, STRAIGHTDIST, Z, X, Y, creek_order_swapped, ID, name)

In [ ]:
# Section 8 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI, reviewed by Sam Kraus 12/20/24

from Process_CrossSections import process_xsects, process_xsects_diagnostic

WIDTH, DEPTH, AREA = process_xsects(creekmask, creek, IDXSEG, ordermax, figview=False)

# # diagnostic version: uncomment and run for debugging
# WIDTH, DEPTH, AREA = process_xsects_diagnostic(creekmask, creek, skeleton, IDXSEG, ordermax, figview=False)

In [ ]:
# Section 9 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI, reviewed by Sam Kraus 12/16/24

from Process_JunctionAngles import process_junction_angles, process_junction_angles_diagnostic

ANGLEORDER = process_junction_angles(skeleton, creekmask, Z, IDXBRANCH, SINUOUSLENGTH, SEGMENTS, fig_view=False)

# # debugging option, uncomment to run:
# ANGLEORDER = process_junction_angles_diagnostic(skeleton, creekmask, Z, IDXBRANCH, SINUOUSLENGTH, SEGMENTS, fig_view=False)

In [ ]:
# Section 10 - CHIROL_CREEK_ALGORITHM_2024.m
# written by Claude AI, reviewed by Sam Kraus 12/23/24

from Save_Results import process_creek_morphometry, process_creek_morphometry_diagnostic

SUMMARY_df, THRESHOLD_df, SINUOUSLENGTH, STRAIGHTDIST, SINUOSITY, DEPTH, WIDTH, AREA = process_creek_morphometry(
                                                                                                                ANGLEORDER, ID, SEGMENTS, SINUOUSLENGTH, 
                                                                                                                SINUOSITY, WIDTH, DEPTH, AREA, STRAIGHTDIST, 
                                                                                                                shortname, Cth, LZth, HZth, Ctharea, LZtharea, HZtharea
                                                                                                                )

# # debugging version, uncomment to run
# SUMMARY_df, THRESHOLD_df, SINUOUSLENGTH, STRAIGHTDIST, SINUOSITY, DEPTH, WIDTH, AREA = process_creek_morphometry_diagnostic(ANGLEORDER, ID, SEGMENTS, SINUOUSLENGTH, 
#                                  SINUOSITY, WIDTH, DEPTH, AREA, STRAIGHTDIST, 
#                                  shortname, Cth, LZth, HZth, Ctharea, LZtharea, HZtharea)